# Implementation of an OLS solver

In this exercise, you will implement your own function for solving OLS regression problems in Python.

The function takes the data samples in matrix-form ($X$, $y$) as inputs and returns the minimizing solution $\beta$ as well as the remaining error $\mathcal{L}(\beta)$.

In [1]:
# Imports
import sys
sys.path.append(r'C:\Users\tscha\BAML\.venv\Lib\site-packages')
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

## Exercise a)

Implement the function.

In [2]:
def fit_parameters(X, y):
    """Compute optimal parameters by least-squares regression.

    Args:
        X (np.ndarray): The input variables, containing intercept variables if required.
        y (np.ndarray): The target variables.

    Returns:
        np.ndarray: The parameter vector (beta)
        float: The remaining loss
    """
    # Solve X^T * X * beta = X^T * y
    # The @ operator performs a matrix multiplication.
    beta = np.linalg.solve(X.T @ X, X.T @ y)
    predicted = X @ beta
    loss = np.square(predicted - y).sum()
    return beta, loss

## Exercise b)

For our provided toy data set (*ols-implementation-data.csv*), find the optimal regression parameters with the help of your implementation. Don't forget to add a variable for the intercept parameter!

In [3]:
# Load dataset
data = pd.read_csv("ols-implementation-data.csv")
X = data[["x1", "x2"]].to_numpy()
y = data["y"].to_numpy()

# Add intercept variables
X_with_intercept = np.hstack([np.ones([X.shape[0], 1]), X])

# Find optimal parameter values
beta, loss = fit_parameters(X_with_intercept, y)
print(f"Parameters by our model: {beta}")
print(f"Loss by our model: {loss:.2f}\n")

Parameters by our model: [47.81880739 -0.25241394  3.38759361]
Loss by our model: 96199.44



## Exercise c)

Repeat b) with the aid of scikit-learn [``LinearRegression``](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) and verify your solution.

In [4]:
# Using scikit-learn
model = LinearRegression(fit_intercept=True)
model.fit(X, y)
beta_skl = np.array([model.intercept_, *model.coef_])
y_pred_skl = model.predict(X)
loss_skl = np.square(y_pred_skl - y).sum()
print(f"Parameters by scikit-learn: {beta_skl}")
print(f"Loss by scikit-learn: {loss_skl:.2f}")

Parameters by scikit-learn: [47.81880739 -0.25241394  3.38759361]
Loss by scikit-learn: 96199.44


## Exercise d)

How much of the total variance can you explain with your model? Compute the R^2 measure. What happens if you forget about the intercept? How does the R^2 measure compare?

In [5]:
# R^2 measure
tss = np.square(y - y.mean()).sum()
rss = np.square(X_with_intercept @ beta - y).sum()
R2 = 1 - rss/tss
print(f"R2 with intercept: {R2:.3f}\n")

R2 with intercept: 0.554



In [6]:
# Without intercept
beta, loss = fit_parameters(X, y)
tss = np.square(y - y.mean()).sum()
rss = np.square(X @ beta - y).sum()
R2 = 1 - rss/tss
print(f"Parameters without intercept: {beta}")
print(f"R2 without intercept: {R2:.3f}\n")

Parameters without intercept: [-0.24122558  4.03669799]
R2 without intercept: -0.485



## Exercise e)

The computed R^2 value is not very good (even with the intercept). What could be the reason?

**Solution**: The model choice could be an inadequate match. Nonlinear transformations of the input variables (i.e. generalized least squares) could provide a better solution. With quadratric terms, the fit is much better:

In [7]:
## Extra: Fitting with generalized linear regression
# We add additional quadratic terms to our model 
X_GLS = np.hstack([X_with_intercept, np.square(X)])
beta, loss = fit_parameters(X_GLS, y)
print(f"Parameters with squared terms: {beta}")
print(f"Loss by our model: {loss:.2f}")

tss = np.square(y - y.mean()).sum()
rss = np.square(X_GLS @ beta - y).sum()
R2 = 1 - rss/tss
print(f"R2 of GLS: {R2:.3f}")

Parameters with squared terms: [ 2.20087707e+01 -1.00142510e+00  3.00182908e+00  2.48549853e-01
  3.75508163e-03]
Loss by our model: 1014.96
R2 of GLS: 0.995
